# W4_Clustering Text Data with Gaussian Mixtures

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import spdiags
from scipy.stats import multivariate_normal
from copy import deepcopy
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans

In [2]:
data = pd.read_csv('people_wiki.csv').head(5000) # 5000 * 3
data.head()

URI                 name  \
0        <http://dbpedia.org/resource/Digby_Morrell>        Digby Morrell   
1       <http://dbpedia.org/resource/Alfred_J._Lewy>       Alfred J. Lewy   
2        <http://dbpedia.org/resource/Harpdog_Brown>        Harpdog Brown   
3  <http://dbpedia.org/resource/Franz_Rottensteiner>  Franz Rottensteiner   
4               <http://dbpedia.org/resource/G-Enka>               G-Enka   

                                                text  
0  digby morrell born 10 october 1979 is a former...  
1  alfred j lewy aka sandy lewy graduated from un...  
2  harpdog brown is a singer and harmonica player...  
3  franz rottensteiner born in waidmannsfeld lowe...  
4  henry krvits born 30 december 1974 in tallinn ...

In [3]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    return csr_matrix( (data, indices, indptr), shape)

tf_idf_matrix = load_sparse_csr('4_tf_idf.npz')
tf_idf_matrix = normalize(tf_idf_matrix)
tf_idf_matrix # sparse matrix with 5000 data points, 100282 features

<5000x100282 sparse matrix of type '<class 'numpy.float64'>'
	with 881415 stored elements in Compressed Sparse Row format>

In [4]:
ser = pd.read_json('4_map_index_to_word.json', typ='series')
map_index_to_word = pd.DataFrame(ser, columns=['word_index'])
map_index_to_word['word'] = map_index_to_word.index
map_index_to_word.index = map_index_to_word['word_index']
map_index_to_word.drop('word_index', axis=1, inplace=True)
map_index_to_word.head()

word
word_index        
93320            0
90689           00
88780          000
14059       000577
29242         0006

In [5]:
# log probability function for diagonal covariance Gaussian
def diag(array):
    n = len(array)
    return spdiags(array, 0, n, n)

def logpdf_diagonal_gaussian(x, mean, cov):
    '''
    Compute logpdf of a multivariate Gaussian distribution with diagonal covariance at a given point x.
    A multivariate Gaussian distribution with a diagonal covariance is equivalent
    to a collection of independent Gaussian random variables.

    x should be a sparse matrix. The logpdf will be computed for each row of x.
    mean and cov should be given as 1D numpy arrays
    mean[i] : mean of i-th variable
    cov[i] : variance of i-th variable
    '''

    n = x.shape[0]
    dim = x.shape[1]
    assert(dim == len(mean) and dim == len(cov))

    scaled_x = x.dot( diag(1./(2*np.sqrt(cov))) )
    scaled_mean = mean/(2*np.sqrt(cov))

    return -np.sum(np.log(np.sqrt(2*np.pi*cov))) - pairwise_distances(scaled_x, [scaled_mean], 'euclidean').flatten()**2

In [6]:
# EM algorithm for sparse data
def log_sum_exp(x, axis):
    '''Compute the log of a sum of exponentials'''
    x_max = np.max(x, axis=axis)
    if axis == 1:
        return x_max + np.log(np.sum(np.exp(x-x_max[:,np.newaxis]), axis=1))
    else:
        return x_max + np.log(np.sum(np.exp(x-x_max), axis=0))

def EM_for_high_dimension(data, means, covs, weights, cov_smoothing=1e-5, maxiter=int(1e3), thresh=1e-4, verbose=False):
    n = data.shape[0]
    dim = data.shape[1]
    mu = deepcopy(means)
    Sigma = deepcopy(covs)
    K = len(mu)
    weights = np.array(weights)

    ll = None
    ll_trace = []

    for i in range(maxiter):
        # E-step: compute responsibilities
        logresp = np.zeros((n,K))
        for k in range(K):
            logresp[:,k] = np.log(weights[k]) + logpdf_diagonal_gaussian(data, mu[k], Sigma[k])
        ll_new = np.sum(log_sum_exp(logresp, axis=1))
        if verbose:
            print(ll_new)
        logresp -= np.vstack(log_sum_exp(logresp, axis=1))
        resp = np.exp(logresp)
        counts = np.sum(resp, axis=0)

        # M-step: update weights, means, covariances
        weights = counts / np.sum(counts)
        for k in range(K):
            mu[k] = (diag(resp[:,k]).dot(data)).sum(axis=0)/counts[k]
            mu[k] = mu[k].A1

            Sigma[k] = diag(resp[:,k]).dot( data.multiply(data)-2*data.dot(diag(mu[k])) ).sum(axis=0) + (mu[k]**2)*counts[k]
            Sigma[k] = Sigma[k].A1 / counts[k] + cov_smoothing*np.ones(dim)

        # check for convergence in log-likelihood
        ll_trace.append(ll_new)
        if ll is not None and (ll_new-ll) < thresh and ll_new > -np.inf:
            ll = ll_new
            break
        else:
            ll = ll_new

    out = {'weights':weights, 'means':mu, 'covs':Sigma, 'loglik':ll_trace, 'resp':resp}

    return out

In [7]:
# initialize mean parameters using k-means
np.random.seed(5)
num_clusters = 25

kmeans_model = KMeans(n_clusters=num_clusters, n_init=5, max_iter=400, random_state=1, n_jobs=-1).fit(tf_idf_matrix)
centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_

means = [centroid for centroid in centroids]

In [8]:
means[0]

array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         2.01954039e-04,   1.04447276e-04,   9.87192763e-05])

In [9]:
# initialize cluster weights
num_docs = tf_idf_matrix.shape[0]
weights = []
for i in range(num_clusters):
    num_assigned = sum(cluster_assignment==i) # compute the number of data points assigned to cluster i
    w = float(num_assigned) / num_docs
    weights.append(w)

In [10]:
# initialize covariances
covs = []
for i in range(num_clusters):
    member_rows = tf_idf_matrix[cluster_assignment==i]
    cov = (member_rows.multiply(member_rows) - 2*member_rows.dot(diag(means[i]))).sum(axis=0).A1 / member_rows.shape[0] + means[i]**2
    cov[cov < 1e-8] = 1e-8
    covs.append(cov)

In [11]:
len(covs[0])

100282

In [12]:
out = EM_for_high_dimension(tf_idf_matrix, means, covs, weights, cov_smoothing=1e-10)
out['loglik'] 

[3855847476.7012835, 4844053202.46348, 4844053202.46348]

In [13]:
def visualize_EM_clusters(tf_idf_matrix, means, covs, map_index_to_word):
    num_clusters = len(means)
    for c in range(num_clusters):
        print('')
        print('==========================================================')
        print('Cluster {0:d}: Largest mean parameters in cluster '.format(c))
        print('{0: <12}{1: <12}{2: <12}'.format('Word', 'Mean', 'Variance'))
        
        # The k'th element of sorted_word_ids should be the index of the word 
        # that has the k'th-largest value in the cluster mean. Hint: Use np.argsort().
        sorted_word_ids = np.argsort(means[c])
    
        for i in sorted_word_ids[-5:]:
            print('{0: <12}{1: <10.2e}{2: <10.2e}'.format(map_index_to_word.loc[i]['word'], means[c][i], covs[c][i]))

In [14]:
visualize_EM_clusters(tf_idf_matrix, out['means'], out['covs'], map_index_to_word)


Cluster 0: Largest mean parameters in cluster 
Word        Mean        Variance    
elected     2.91e-02  8.95e-04  
liberal     2.93e-02  4.55e-03  
party       5.89e-02  2.61e-03  
election    5.89e-02  3.21e-03  
minister    7.57e-02  7.42e-03  

Cluster 1: Largest mean parameters in cluster 
Word        Mean        Variance    
directed    3.39e-02  2.22e-03  
feature     3.69e-02  1.81e-03  
festival    4.66e-02  3.60e-03  
films       5.50e-02  2.97e-03  
film        1.76e-01  6.07e-03  

Cluster 2: Largest mean parameters in cluster 
Word        Mean        Variance    
design      3.20e-02  4.59e-03  
artist      3.61e-02  1.44e-03  
gallery     3.65e-02  3.40e-03  
museum      5.62e-02  7.27e-03  
art         1.26e-01  6.83e-03  

Cluster 3: Largest mean parameters in cluster 
Word        Mean        Variance    
team        4.68e-02  1.30e-03  
coach       5.57e-02  5.91e-03  
points      6.25e-02  5.92e-03  
nba         1.01e-01  1.22e-02  
basketball  1.86e-01  7.78e-03  


In [15]:
# compare to random initialization
np.random.seed(5)
num_clusters = len(means)
num_docs, num_words = tf_idf_matrix.shape

random_means = []
random_covs = []
random_weights = []

for k in range(num_clusters):
    
    # create a numpy array of length num_words with random normally distributed values
    mean = np.random.normal(0.0, 1.0, num_words)
    # create a numpy array of length num_words with random values uniformly distributed between 1 and 5.
    cov = np.random.uniform(1.0, 5.0, num_words)
    # initially give each cluster equal weight
    weight = 1 / num_clusters

    random_means.append(mean)
    random_covs.append(cov)
    random_weights.append(weight)

In [16]:
random_init_out = EM_for_high_dimension(tf_idf_matrix, random_means, random_covs, random_weights, cov_smoothing=1e-5)

In [17]:
random_init_out['loglik']

[-764085987.57300639,
 2282866699.1732855,
 2362585588.3564939,
 2362875609.1670547,
 2362875609.1670547]

In [18]:
visualize_EM_clusters(tf_idf_matrix, random_init_out['means'], random_init_out['covs'], map_index_to_word)


Cluster 0: Largest mean parameters in cluster 
Word        Mean        Variance    
bbc         1.20e-02  1.82e-03  
singapore   1.80e-02  5.72e-03  
music       2.03e-02  2.37e-03  
her         2.63e-02  1.82e-03  
she         4.21e-02  5.79e-03  

Cluster 1: Largest mean parameters in cluster 
Word        Mean        Variance    
league      1.04e-02  9.58e-04  
music       1.05e-02  9.94e-04  
university  1.06e-02  3.19e-04  
she         1.29e-02  1.67e-03  
he          1.38e-02  1.11e-04  

Cluster 2: Largest mean parameters in cluster 
Word        Mean        Variance    
festival    1.06e-02  2.02e-03  
he          1.13e-02  1.20e-04  
music       1.46e-02  1.42e-03  
her         2.43e-02  2.57e-03  
she         3.30e-02  3.96e-03  

Cluster 3: Largest mean parameters in cluster 
Word        Mean        Variance    
he          1.00e-02  9.16e-05  
series      1.04e-02  5.20e-04  
film        1.43e-02  2.01e-03  
her         1.76e-02  1.50e-03  
she         2.69e-02  3.29e-03  
